In [1]:
import pandas as pd
import requests
import json
import logging
from bs4 import BeautifulSoup
import time

In [2]:
# GET TOP TOURISM COUNTRIES
df_tourism = pd.read_csv('./data/most_traveled_countries.csv')
# SOURCE: http://worldpopulationreview.com/countries/most-visited-countries/
# df_tourism.head()

top20 = list(df_tourism.sort_values(by='touristArrivals', ascending=False)['name'][:20])
print(top20)

['France', 'United States', 'Spain', 'China', 'Italy', 'United Kingdom', 'Germany', 'Mexico', 'Thailand', 'Turkey', 'Austria', 'Malaysia', 'Hong Kong', 'Greece', 'Russia', 'Japan', 'Canada', 'Portugal', 'Saudi Arabia', 'Poland']


In [3]:
# GET COUNTRIES
country_names = [c_name for c_name in pd.read_csv('./data/country_names.csv')['name']]
# SOURCES: http://worldpopulationreview.com/countries/most-visited-countries/
    # & https://www.travel-advisory.info/api


In [4]:
len(country_names)

178

In [5]:
# GET ALL URLS FOR 20 COUNTRIES
articles = []
for country in country_names:
#     print(f'Searching {country}...')
#     for n in range(1, 10):
    try:
        url_base = f'https://ordinarytraveler.com/category/{country}'
        res_ = requests.get(url_base)
        soup_ = BeautifulSoup(res_.content, 'lxml')
        time.sleep(1)

        for link in soup_.find_all(attrs={'class': 'entry-title-link'}):
            article = {}
            article['href'] = link.attrs['href']
            article['title'] = link.text.strip()
            article['country'] = country
            articles.append(article)
    except:
        print(f'Error with {country}.')

df = pd.DataFrame(articles)
# df.to_csv('./data/all_articles.csv')
print('Shape of dataframe:', df.shape)
df.head()

Shape of dataframe: (93, 3)


,href,title,country
0,https://ordinarytraveler.com/chasing-waves-in-...,Budget Tips for Taking a Surf Trip Along The E...,Australia
1,https://ordinarytraveler.com/top-things-to-do-...,10 Best Things To Do In Aruba,Aruba
2,https://ordinarytraveler.com/where-to-stay-aru...,Pristine Beaches and Warm Hospitality at Aruba...,Aruba
3,https://ordinarytraveler.com/homemade-tsatsiki...,Easy Homemade Tzatziki Sauce Recipe,Belgium
4,https://ordinarytraveler.com/antwerp-belgium-t...,"The Best Things To Do In Antwerp, Belgium",Belgium


In [6]:
content_filed = []
for url in df['href']:
    res_post = requests.get(url)
    soup_post = BeautifulSoup(res_post.content, 'lxml')

    paragraphs = soup_post.find_all(attrs={'class': 'entry-content'})

    text = ' '.join([paragraph.text.replace('\n', '') for paragraph in paragraphs])
    content_filed.append(text)
    
    time.sleep(1)
df['text'] = content_filed

In [7]:
df.tail()

,href,title,country,text
88,https://ordinarytraveler.com/an-island-fit-for...,Koh Phangan – It’s More Than Just a Party Island,Thailand,Koh Phangan has been a popular destination thr...
89,https://ordinarytraveler.com/head-highs-banana...,"Head Highs, Banana Pancakes & Gorgeous Islands...",Thailand,Since traveling through Australia felt like we...
90,https://ordinarytraveler.com/the-kiteboarding-...,Mui Ne Travel Guide: The Kiteboarding Mecca of...,Vietnam,This Mui Ne travel guide covers all of the ess...
91,https://ordinarytraveler.com/hoi-an-vietnam,"Hoi An, Vietnam Travel Guide",Vietnam,"Hoi An, Vietnam Travel GuideHoi An is a place..."
92,https://ordinarytraveler.com/three-million-mot...,"Northern Vietnam Travel Guide – Hanoi, Ha Long...",Vietnam,Our Northern Vietnam Travel Guide provides tip...


In [8]:
df.to_csv('./data/articles_ordinarytraveler.csv', index=False)